In [1]:
!pip install pandas sqlalchemy psycopg2-binary requests

In [13]:
!pip install psycopg2

In [14]:
!pip install sqlalchemy-redshift
!pip install redshift_connector

  Obtaining dependency information for sqlalchemy-redshift from https://files.pythonhosted.org/packages/17/8b/2c7201607a9a5d997ace6ca907d1dea9271fd399560b082d3432a72046af/sqlalchemy_redshift-0.8.14-py2.py3-none-any.whl.metadata
  Obtaining dependency information for redshift_connector from https://files.pythonhosted.org/packages/d5/9e/eae3a4fc7c0e90b31abb9e268b445667290e5e45ec33f75bf294c6ed8e07/redshift_connector-2.1.1-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/66.8 kB ? eta -:--:--
     ------ --------------------------------- 10.2/66.8 kB ? eta -:--:--
     ----------------- -------------------- 30.7/66.8 kB 262.6 kB/s eta 0:00:01
     ---------------------------------- --- 61.4/66.8 kB 409.6 kB/s eta 0:00:01
     -------------------------------------- 66.8/66.8 kB 452.8 kB/s eta 0:00:00
  Obtaining dependency information for scramp<1.5.0,>=1.2.0 from https://files.pythonhosted.org/packages/d9/9f/8b2f2749ccfbe4fcef08650896ac47ed919ff25b7ac57b7a1ae7da1

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.5.0 requires botocore<1.29.77,>=1.29.76, but you have botocore 1.34.120 which is incompatible.


In [15]:
import requests
import pandas as pd
from sqlalchemy import create_engine
import psycopg2

In [16]:
# Configuración de las credenciales de Redshift
redshift_user = 'camilo_lyv_coderhouse'
redshift_password = 'cC03egF87w'
redshift_endpoint = 'data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com'
redshift_db = 'data-engineer-database'
redshift_port = 5439

In [17]:
# Verificación de las credenciales de Redshift
try:
    engine = create_engine(f'redshift+psycopg2://{redshift_user}:{redshift_password}@{redshift_endpoint}:{redshift_port}/{redshift_db}')
    print("Conexión a Redshift establecida correctamente.")
except Exception as e:
    print("Error al establecer la conexión a Redshift:", str(e))

Conexión a Redshift establecida correctamente.


In [18]:
# Extraer datos de la API
url = "https://api-colombia.com/api/v1/Department"
try:
    response = requests.get(url)
    data = response.json()
    print("Datos extraídos de la API correctamente.")
except Exception as e:
    print("Error al extraer datos de la API:", str(e))
    data = None

Datos extraídos de la API correctamente.


In [19]:
# Convertir a DataFrame de pandas
try:
    df = pd.json_normalize(data)
    print("Datos convertidos a DataFrame correctamente.")
except Exception as e:
    print("Error al convertir datos a DataFrame:", str(e))
    df = None

Datos convertidos a DataFrame correctamente.


In [22]:
# Eliminar la columna 'description'
df = df.drop(columns=['description'])

In [23]:
# Carga y verificacion de datos en la tabla redshift
if df is not None:
    try:
        df.to_sql('departamentos', engine, if_exists='replace', index=False)
        print("Datos cargados en la tabla 'departamentos' de Redshift correctamente.")
    except Exception as e:
        print("Error al cargar datos en Redshift:", str(e))

Datos cargados en la tabla 'departamentos' de Redshift correctamente.


In [24]:
# Verificamos los datos en la tabla en redshift a traves de una consulta SQL
try:
    with engine.connect() as connection:
        result = connection.execute("SELECT * FROM departamentos LIMIT 5")
        print("Primeros 5 registros en la tabla 'departamentos' de Redshift:")
        for row in result:
            print(row)
except Exception as e:
    print("Error al verificar los datos en Redshift:", str(e))

Primeros 5 registros en la tabla 'departamentos' de Redshift:
(10, 'Casanare', 380, 19, 44640, 442068, '8', 1, None, None, None, 3, None, None, None, None, None)
(11, 'Cauca', 399, 41, 29308, 1516018, '2', 1, None, None, None, 2, None, None, None, None, None)
(12, 'Cesar', 441, 25, 22905, 1341697, '4', 1, None, None, None, 1, None, None, None, None, None)
(13, 'Chocó', 467, 31, 46530, 553519, '4', 1, None, None, None, 2, None, None, None, None, None)
(14, 'Córdoba', 498, 30, 25020, 1856496, '4', 1, None, None, None, 1, None, None, None, None, None)
